Mount Google Drive and Load Armenian Text File

In [ ]:
from google.colab import drive
import os
import re
import nltk
from collections import Counter

drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/NLP_2/lab 1/merged_text.txt"

with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

# Remove non-Armenian characters (but keep spaces)
text = re.sub(r'[^\u0530-\u058F\s]', '', text)

# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
tokens = nltk.word_tokenize(text)

print("Sample tokens:", tokens[:10])

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Sample tokens: ['ՀՀ', 'Կենտրոնական', 'բանկն', 'արժույթի', 'շուկայում', 'իրականացրած', 'ամենօրյա', 'միջամտությունների', 'ընթացքում', 'այս']


Prepare Data for RNN

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Build vocabulary
token_counts = Counter(tokens)
vocab = {word: idx for idx, word in enumerate(set(tokens))}
token_to_index = {token: idx for idx, token in enumerate(vocab)}
index_to_token = {idx: token for token, idx in token_to_index.items()}

# Prepare sequences
window_size = 50
sequences = []
targets = []

for i in range(len(tokens) - window_size):
    seq = tokens[i:i + window_size]
    target = tokens[i + window_size]
    sequences.append([token_to_index[token] for token in seq])
    targets.append(token_to_index[target])

X = np.array(sequences)
y = np.array(targets)

print("Vocabulary size:", len(vocab))
print("Number of sequences:", len(sequences))
print("First sequence:", sequences[0])
print("First target:", targets[0])

Vocabulary size: 113690
Number of sequences: 1368311
First sequence: [74792, 111617, 32100, 69887, 25071, 105796, 27186, 16725, 2327, 17274, 4350, 61817, 43766, 80023, 13655, 79811, 78210, 65284, 55042, 69983, 65284, 55042, 19473, 19596, 78210, 74792, 47788, 98355, 32125, 16667, 13634, 36845, 14824, 6773, 48089, 68220, 3481, 78210, 45237, 3805, 89475, 78303, 43346, 68220, 45237, 3805, 92669, 74792, 47788, 61817]
First target: 62775


RNN Model and Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_tensor = torch.tensor(X, dtype=torch.long).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

# Create DataLoader for batching
BATCH_SIZE = 64
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

embed_size = 128
hidden_size = 128
output_size = len(vocab)
learning_rate = 0.001
epochs = 5

model = RNNModel(len(vocab), embed_size, hidden_size, output_size).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Using device: cuda


In [ ]:
# Training Loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        # Forward pass
        output = model(X_batch)
        loss = loss_fn(output, y_batch)
        epoch_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

Epoch 1/5, Loss: 7.8655
Epoch 2/5, Loss: 6.8456
Epoch 3/5, Loss: 6.4539
Epoch 4/5, Loss: 6.1463
Epoch 5/5, Loss: 5.8699


Armenian Text Generation

In [ ]:
def generate_text(seed_text, num_words=100):
    model.eval()
    words = seed_text.split()

    for _ in range(num_words):
        input_seq = words[-49:]
        input_indices = [token_to_index.get(word, 0) for word in input_seq]
        input_tensor = torch.tensor([input_indices], dtype=torch.long).to(device)

        # Predict next word
        with torch.no_grad():
            output = model(input_tensor)
            _, predicted_idx = torch.max(output, dim=1)
            predicted_word = index_to_token[predicted_idx.item()]

        words.append(predicted_word)

    return ' '.join(words)

# Example usage
seed_text = "Բարև ձեզ"
generated_text = generate_text(seed_text, num_words=50)
print(generated_text)

Բարև ձեզ է եղել նաև որ այս պահին Գորիսի Սդղի գյոլ կոչվող տարածքում բոլոր ավտոճանապարհները հիմնականում անցանելի են Դժվարանցանելի է ՍոթքՔարվաճառ ավտոճանապարհը Վրաստանի ՆԳՆ ԱԻ դեպարտամենտից ստացված տեղեկատվության համաձայն ՍտեփանծմինդաԼարս ավտոճանապարհը բաց է բոլոր տեսակի տրանսպորտային միջոցների համար ՀՀ ԱԻՆ հիդրոօդերևութաբանության և մոնիտորինգի պետական ծառայություն Հանրապետության տարածքում Նոյեմբերի ին սպասվում է առանց
